ReLU activation function is one of the breakthrought in deep neural network due to its simplcity and the fact that its locals derivative equals to $1$ for non-negative value, hence prevent the vanishing gradient problem. Despite it efectiveness and simplicity, some effort on make ReLU more expressive has been made. Some effort to  generalize  ReLU are based on using a nonzero slopes $\alpha$ for negative input. As for November 2020, there are 3 variant of ReLU that do this kind of generalization. Absolute value rectification fixes $\alpha =-1$. [LeakyReLU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) fixes $\alpha$ to a small value, while [PReLU](https://arxiv.org/pdf/1502.01852.pdf) treats $\alpha$ as a learnable parameter.

Let us begin by the formal definition of Dynamic ReLU based on the [paper](https://arxiv.org/pdf/2003.10027.pdf)


**Definition** : Let  us  denote  the  traditional  or  static  ReLU  as $\textbf{y}=  max\{\textbf{x},0\}$, where $\textbf{x}$ is the input vector. For the input $\textbf{x}_c$ at the $c$ thchannel, the activation is  computed  as $y_c=  max\{x_c,0\}$. ReLU  can  be  generalized  to  a  parametric piecewise linear function $y_c= max_k\{a_k^c(x_c)+b_k^c\}$. Dynamic ReLU further extend this piecewise linear function from static to dynamic by adapting $a_k^c$, $b_k^c$ based upon all input elements $x=\{x_c\}$ as follows: $y_c=f_{\theta(x)}(x_c) =   max_{1≤k≤K}\{a^k_c(\mathbf{x})x_c+b^k_c(\mathbf{x})\}$where the coefficients $(a^k_c,b^k_c)$ are the output of a hyper function $\theta(\mathbf{x})$ as:$[a^1_1,...,a^1_C,...,a^K_1,...,a^K_C,b^1_1,...,b^1_C,...,b^K_1,\dots,b^K_C]^{\mathsf{T}}=\theta(\mathbf{x})$,where $K$ is the number of functions, and $C$ is the number of channels.

Let us rewrite vanila ReLU $\textbf{y}=  max\{\textbf{x},0\}$ to a generalized form from the definition above. It is easy to show that $K=2$ for this vanila ReLU and $\theta(\mathbf{x}) =[a^1_c(x), b^1_c(x) , a^2_c(x) , b^2_c(x)]^{\mathsf{T}} = [1,0,0,0]^{\mathsf{T}}$. Let us plug in some number to $y_c=f_{\theta(x)}(x_c)$. Let $x_c=-1$

$f_{\theta(x)}(-1) =   max_{1≤k≤2}\{a^k_c(\mathbf{x})(-1)+b^k_c(\mathbf{x})\}=a^2_c(\mathbf{x})(-1)+b^2_c(\mathbf{x})=0(-1)+0=0$


